# Graphing Notebook for Neural Multicompartmental Simulator


Created by EF Shorer
May 2021
MSc(Med) Computational Neuroscience


This notebook is designed to accept an HDF5 file from a neural simulation as input and then display relevant graphs. 
The HDF5 File needs to located in the same directory as this notebook


## Import relevant classes


In [2]:
try:
        
    import h5py
    
    from ipywidgets import widgets, Layout, interact, interactive, interactive_output, Dropdown
    from IPython.display import display
    import bqplot as bqp
    from bqplot import pyplot as plt # bqplot is a plotting for jupyter
    import matplotlib.pyplot as mplt
  
    import numpy as np
    import random
    import seaborn as sns 
    import pandas as pd
    import graphing as gr
    #!pip install viola
    ########## Modules required for artist drawing on matplotlib
    import matplotlib.path as mpath
    import matplotlib.lines as mlines
    import matplotlib.patches as mpatches
    from matplotlib.collections import PatchCollection
    
      
except ModuleNotFoundError:
        print("A class you are trying to import is not present in the directory")
    
except Exception:
        print("Something went wrong - modules have not been imported")

else: 
    print("All relevant classes imported")
    
    

All relevant classes imported


## Select relevant HDF5 File


In [62]:
#######################################################
#### GUI 

edt_filename = widgets.Text(description = 'File name', value='test2')
btn_select = widgets.Button(description = 'Select file',button_style='success')
output_file = widgets.Output()

display(edt_filename, btn_select,output_file)



#######################################################
#### FUNCTIONS

def btn_select_clicked(b):
    
    file_name = "\\ "
    file_name = file_name + edt_filename.value 
    try: 
        with h5py.File(file_name, mode='r') as hdf:
            global C
            C = hdf.get('COMPARTMENTS')
            C_group_arr =[]
            t_arr_bool = True
          
            global master_arr, t_arr
            master_arr = []
            t_arr =[]
            
            print("File found and content loaded into memory")
    except: 
        raise("File not found")
        raise("File needs to be in base directory")


btn_select.on_click(btn_select_clicked)

Text(value='test2', description='File name')

Button(button_style='success', description='Select file', style=ButtonStyle())

Output()

File found and content loaded into memory
File found and content loaded into memory


# Graphing

## A) Plot compartment parameters

In [66]:
print(C.items)






    
        

<bound method MappingHDF5.items of <Closed HDF5 group>>
